In [1]:
'''

啟動伺服器基本樣版

設定業務功能
    當用戶關注時
        記錄個資，綁定菜單，回傳文字消息與圖片消息，介紹功能與使用方法
    當用戶點擊自定義菜單或輸入文字消息時
        傳送圖片消息或相應文字消息。
    當用戶點擊按鈕消息的時候
        會將用戶個資紀錄回檔案內，以供未來聯繫用

'''

'\n\n啟動伺服器基本樣版\n\n設定業務功能\n    當用戶關注時\n        記錄個資，綁定菜單，回傳文字消息與圖片消息，介紹功能與使用方法\n    當用戶點擊自定義菜單或輸入文字消息時\n        傳送圖片消息或相應文字消息。\n    當用戶點擊按鈕消息的時候\n        會將用戶個資紀錄回檔案內，以供未來聯繫用\n\n'

In [2]:
"""

啟用伺服器基本樣板

"""

# 引用Web Server套件
from flask import Flask, request, abort

# 從linebot 套件包裡引用 LineBotApi 與 WebhookHandler 類別
from linebot import (
    LineBotApi, WebhookHandler
)

# 引用無效簽章錯誤
from linebot.exceptions import (
    InvalidSignatureError
)

# 載入json處理套件
import json

# 載入基礎設定檔
secretFileContentJson=json.load(open("../line_secret_key",'r',encoding="utf-8"))
server_url=secretFileContentJson.get("server_url")

# 設定Server啟用細節
app = Flask(__name__,static_url_path = "/images" , static_folder = "../images/")

# 生成實體物件
line_bot_api = LineBotApi(secretFileContentJson.get("channel_access_token"))
handler = WebhookHandler(secretFileContentJson.get("secret_key"))

# 啟動server對外接口，使Line能丟消息進來
@app.route("/", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)

    return 'OK'

In [3]:
"""


當用戶關注Line@後，Line會發一個FollowEvent，

我們接受到之後，取得用戶個資，對用戶綁定自定義菜單，會回傳四個消息給用戶


"""

'\n\n\n當用戶關注Line@後，Line會發一個FollowEvent，\n\n我們接受到之後，取得用戶個資，對用戶綁定自定義菜單，會回傳四個消息給用戶\n\n\n'

In [4]:
'''

製作文字與圖片的教學訊息

'''
# 將消息模型，文字收取消息與文字寄發消息 引入
from linebot.models import (
    MessageEvent, TextMessage, TextSendMessage, ImageSendMessage
)

# 消息清單
reply_message_list = [
    TextSendMessage(text="歡迎來到PM2.5智慧特助"),
]
text_reply_message1 = TextSendMessage(text="在空氣中有許多不同的物質，包含氧氣 (O2)、水蒸氣 (H2O)等等一些無害的物質，其中足以危害健康或生活環境的物質，我們稱之為空氣汙染物。其中漂浮在空氣中像灰塵的粒狀物稱為懸浮微粒 (particulate matter, PM)。")
text_reply_message2 = TextSendMessage(text="PM粒徑大小有分別，小於或等於2.5微米(μm; 為1公尺的百萬分之一)的粒子，就稱為PM2.5，通稱細懸浮微粒，其單位以微克/立方公尺(μg/m3)，PM2.5的直徑還不到人的頭髮絲粗細的1/28，非常微細。")
text_reply_message3 = TextSendMessage(text="比較大的懸浮粒子可以被人體纖毛和黏液自行過濾掉，無法通過我們的鼻子和咽喉，進入到人體內部。然而，小於10微米的細懸浮，包含PM2.5，可以通過這些屏障到達支氣管和肺泡所在的位置並沉積於體內，在無法被清除時，便會造成各種疾病，特別是PM2.5以下的微粒對人體健康的影響最大。")
text_reply_message4 = TextSendMessage(text="1.空品監測：即刻知道目前桃園空氣品質\n\n2.空品預報：告訴您桃園市下個時段可能的空氣品質\n\n3.空品科普：讓您擁有PM2.5的小知識\n\n4.觀光導覽：桃園市哪裡好玩好食一次推薦給您\n\n5.天氣預報：接下來天氣如何呢？點擊我馬上讓您知道")
text_reply_message5 = TextSendMessage(text="1.PM2.5桃園預測值：13.7\n\n2.PM2.5平鎮預測值：13.9\n\n3.PM2.5大園預測值：14.2\n\n4.PM2.5中壢預測值：19.5\n\n5.PM2.5龍潭預測值：15.2\n\n6.PM2.5觀音預測值：17.1")


In [5]:
'''

建立圖片消息素材

'''

# 將消息模型，文字收取消息與文字寄發消息 引入
from linebot.models import (
    ImageSendMessage
)

# 圖片消息的基本建構特徵
image_message = ImageSendMessage(
    original_content_url='https://%s/images/manual.png' % server_url,
    preview_image_url='https://%s/images/manual.png' % server_url
)
image_message2 = ImageSendMessage(
    original_content_url='https://%s/images/knowledge.png' % server_url,
    preview_image_url='https://%s/images/knowledge.png' % server_url
)

In [6]:
'''
Button篇
    設定模板消息，指定其參數細節。

'''


#引入所需要的消息與模板消息
from linebot.models import (
    MessageEvent, TemplateSendMessage , PostbackEvent
)

#引入按鍵模板
from linebot.models.template import(
    ButtonsTemplate
)

# #引入模板消息的可用行為
# from linebot.models.template import(
#     PostbackTemplateAction,
#     MessageTemplateAction,
#     URITemplateAction,
#     DatetimePickerTemplateAction
# )


'''
alt_text: Line簡覽視窗所出現的說明文字
template: 所使用的模板
ButtonsTemplate: 按鍵模板
    thumbnail_image_url: 展示圖片
    title: 標題
    text: 說明文字
    actions: 模板行為所使用的行為
    data: 觸發postback後用戶回傳值，可以對其做商業邏輯處理

'''
buttons_template_message = TemplateSendMessage(
    alt_text='Buttons template',
    template=ButtonsTemplate(
        thumbnail_image_url='https://i.ytimg.com/vi/dN8fhS8Y3-0/maxresdefault.jpg',
        title='空品科普',
        text='PM2.5到底是什麼呢',
        actions=[
          {
            "type": "message",
            "label": "空氣污染物的小常識",
            "text": "[::text:]空氣污染物的小常識"
          },
          {
            "type": "message",
            "label": "來聽我說關於PM2.5",
            "text": "[::text:]來聽你說關於PM2.5"
          },
          {
            "type": "message",
            "label": "PM2.5與我們健康的關係",
            "text": "[::text:]PM2.5與我們健康的關係"
          }
        ],
)
)

In [7]:
'''
將素材消息做成一本字典，當用戶發出相應消息時，可從此進行查找動作。
'''
template_message_dict = {
    "[::text:]新手加入": reply_message_list,
    "[::text:]空氣污染物的小常識":text_reply_message1,
    "[::text:]來聽你說關於PM2.5":text_reply_message2,
    "[::text:]PM2.5與我們健康的關係":text_reply_message3,
    "[::text:]使用說明":text_reply_message4 ,
    "[::text:]空品科普":buttons_template_message ,
    "[::text:]空品預報":text_reply_message5
    
}

In [8]:
'''

撰寫用戶關注時，我們要處理的商業邏輯

1. 取得用戶個資，並存回伺服器
2. 把先前製作好的自定義菜單，與用戶做綁定
3. 回應用戶，歡迎用的文字消息與圖片消息

'''


# 載入Follow事件
from linebot.models.events import (
    FollowEvent
)

# 載入requests套件
import requests


# 告知handler，如果收到FollowEvent，則做下面的方法處理
@handler.add(FollowEvent)
def reply_text_and_get_user_profile(event):
    
    # 取出消息內User的資料
    user_profile = line_bot_api.get_profile(event.source.user_id)
        
     # 將用戶資訊存在檔案內
    with open("../users.txt", "a") as myfile:
        myfile.write(json.dumps(vars(user_profile),sort_keys=True))
        myfile.write('\r\n')
        
        
    # 將菜單綁定在用戶身上
    linkRichMenuId=secretFileContentJson.get("rich_menu_id")
    linkResult=line_bot_api.link_rich_menu_to_user(secretFileContentJson["self_user_id"], linkRichMenuId)
    
    # 回覆文字消息與圖片消息
    line_bot_api.reply_message(
        event.reply_token,
        reply_message_list
    )

In [9]:
'''

當用戶發出文字消息時，判斷文字內容是否包含[::text:]，
    若有，則從template_message_dict 內找出相關訊息
    若無，則回傳預設訊息。

'''

# 用戶發出文字消息時， 按條件內容, 回傳文字消息
@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    
    if(event.message.text.find('::text:')!= -1):
        line_bot_api.reply_message(
        event.reply_token,
        template_message_dict.get(event.message.text))
    else:
        line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage(text="幹我有說你可以打字嗎"))

In [10]:
#用戶點擊button後，觸發postback event，對其回傳做相對應處理
@handler.add(PostbackEvent)
def handle_post_message(event):
    user_profile = line_bot_api.get_profile(event.source.user_id)
    if (event.postback.data.find('Data1')== 0):
        with open("../user_profile_business.txt", "a") as myfile:
            myfile.write(json.dumps(vars(user_profile),sort_keys=True))
            myfile.write('\r\n')
            line_bot_api.reply_message(
            event.reply_token,
                TextMessage(
                    text='請稍待，會有專人與您聯繫'
                )
            )
    elif (event.postback.data.find('Data2') == 0):
        with open("../user_profile_tutorial.txt", "a") as myfile:
            myfile.write(json.dumps(vars(user_profile),sort_keys=True))
            myfile.write('\r\n')
            line_bot_api.reply_message(
            event.reply_token,
                TextMessage(
                    text='請稍待，我們會派專家與您聯繫'
                )
            )
    else:
        pass

In [ ]:
'''

執行此句，啟動Server，觀察後，按左上方塊，停用Server

'''

if __name__ == "__main__":
    app.run(host='0.0.0.0')

 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
172.17.0.3 - - [03/Jan/2020 08:22:28] "POST / HTTP/1.1" 200 -
172.17.0.3 - - [03/Jan/2020 08:22:32] "POST / HTTP/1.1" 200 -
172.17.0.3 - - [03/Jan/2020 08:22:33] "POST / HTTP/1.1" 200 -
172.17.0.3 - - [03/Jan/2020 08:22:35] "POST / HTTP/1.1" 200 -
172.17.0.3 - - [03/Jan/2020 08:33:32] "POST / HTTP/1.1" 200 -
172.17.0.3 - - [03/Jan/2020 08:33:33] "POST / HTTP/1.1" 200 -
172.17.0.3 - - [03/Jan/2020 08:33:39] "POST / HTTP/1.1" 200 -
172.17.0.3 - - [03/Jan/2020 09:29:02] "POST / HTTP/1.1" 200 -
172.17.0.3 - - [03/Jan/2020 10:41:29] "POST / HTTP/1.1" 200 -
172.17.0.3 - - [03/Jan/2020 10:42:07] "POST / HTTP/1.1" 200 -
172.17.0.3 - - [03/Jan/2020 10:42:31] "POST / HTTP/1.1" 200 -
172.17.0.3 - - [03/Jan/2020 10:43:12] "POST / HTTP/1.1" 200 -
172.17.0.3 - - [03/Jan/2020 11:42:04] "POST / HTTP/1.1" 200 -
172.17.0.3 - - [03/Jan/2020 11:42:10] "POST / HTTP/1.1" 200 -
172.17.0.3 - - [03/Jan/2020 11:42:15] "POST / HTTP/1.1" 200 -
172.17.0.3 -